<a href="https://colab.research.google.com/github/Aryanupadhyay23/Deep-Learning-/blob/main/Callbacks_in_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import kagglehub

path = kagglehub.dataset_download("gowtamsingulur/rvlcdip")
print(path)

Download already complete (4658501240 bytes).
Extracting files...
/root/.cache/kagglehub/datasets/gowtamsingulur/rvlcdip/versions/1


In [6]:
import os

print("Dataset downloaded to:", path)
print(os.listdir(path))


Dataset downloaded to: /root/.cache/kagglehub/datasets/gowtamsingulur/rvlcdip/versions/1
['labels_final.csv', 'data_final']


In [2]:
import os
import numpy as np
import pandas as pd
import random as rn

import tensorflow as tf
import keras

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential

from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers


In [8]:
dir_path = '/root/.cache/kagglehub/datasets/gowtamsingulur/rvlcdip/versions/1'
os.listdir(dir_path)

['labels_final.csv', 'data_final']

In [9]:
train_df = pd.read_csv('/root/.cache/kagglehub/datasets/gowtamsingulur/rvlcdip/versions/1/labels_final.csv')
train_df.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [10]:
train_df['path'].head()

,path
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif
1,imagesl/l/x/t/lxt19d00/502213303.tif
2,imagesx/x/e/d/xed05a00/2075325674.tif
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif
4,imagesq/q/z/k/qzk17e00/2031320195.tif


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load and display a .tif image
image_path = '//root/.cache/kagglehub/datasets/gowtamsingulur/rvlcdip/versions/1/data_final/imagesv/v/o/h/voh71d00/509132755+-2755.tif'
image = Image.open(image_path)

plt.imshow(image)
plt.axis('off')  # Turn off axis labels
plt.show()


In [ ]:
import cv2

img = cv2.imread(image_path)

# If the image is loaded successfully, print its pixel values
if img is not None:
    print(img)
    print(img.shape)
else:
    print("The image could not be loaded. Please check the path and file permissions.")


In [11]:
train_df['label'].value_counts()

,count
label,
0,3016
13,3007
14,3006
12,3006
3,3005
8,3003
9,3002
10,3002
7,3000


In [12]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    validation_split = 0.2
)

In [24]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/root/.cache/kagglehub/datasets/gowtamsingulur/rvlcdip/versions/1/data_final/',
    x_col='path',
    y_col='label',
    subset='training',
    batch_size=100,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224, 224)
)

Found 38400 validated image filenames.


In [25]:
images, labels = next(train_generator)
print(len(images), len(labels))

100 100


In [15]:
print(labels)

[]


In [26]:
valid_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='//root/.cache/kagglehub/datasets/gowtamsingulur/rvlcdip/versions/1/data_final/',
    x_col='path',
    y_col='label',
    subset='validation',
    batch_size=100,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224, 224)
)

Found 9600 validated image filenames.


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax'),  # 16 probability of all 16 classes
])

In [31]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,178,256 (42.64 MB)

 Trainable params: 11,178,256 (42.64 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [33]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard

import os
import datetime

In [34]:
checkpoint_path = "kaggle/working/model_checkpoints/weights.{epoch:02d}-{val_loss:2f}.keras"
log_dir = "/kaggle/working/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [35]:
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.0001
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

board = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)

callbacks = [model_checkpoint, reduce_lr, early_stop, board]


In [36]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2216 - loss: 2.4885
Epoch 1: val_loss improved from inf to 1.91617, saving model to kaggle/working/model_checkpoints/weights.01-1.916169.keras
384/384 ━━━━━━━━━━━━━━━━━━━━ 787s 2s/step - accuracy: 0.2218 - loss: 2.4878 - val_accuracy: 0.4054 - val_loss: 1.9162 - learning_rate: 0.0010
Epoch 2/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4445 - loss: 1.7853
Epoch 2: val_loss improved from 1.91617 to 1.54921, saving model to kaggle/working/model_checkpoints/weights.02-1.549213.keras
384/384 ━━━━━━━━━━━━━━━━━━━━ 756s 2s/step - accuracy: 0.4446 - loss: 1.7851 - val_accuracy: 0.5177 - val_loss: 1.5492 - learning_rate: 0.0010
Epoch 3/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5363 - loss: 1.5078
Epoch 3: val_loss improved from 1.54921 to 1.40473, saving model to kaggle/working/model_checkpoints/weights.03-1.404733.keras
384/384 ━━━━━━━━━━━━━━━━━━━━ 750s 2s/step - accuracy: 0.5363 - loss: 1.5077 - val_ac

KeyboardInterrupt: 